

***AWID dataset***


In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

**Mounting G drive**


> You need to authenticate to GD in order to get the AWID dataset with the scripts.



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Get the resources**

> Get all necessary resources from Google Drive, this works after the authentification.these resources are compressed to save some time.




> The headers where added to these two files AWID-CLS-R-Trn_with_headers.zip and AWID-CLS-R-Tst_with_headers.zip with the function *add_headers*


```
def add_header(headers_path, input_filename, output_filename):
    headers = []

    # Read the data
    data = pd.read_csv(input_filename, sep=',',low_memory=False)
    # create the column names
    with open(headers_path) as cols:
        for line_num, col_name in enumerate(cols):
            headers.append(col_name.rstrip())

    # Set the column headers to the names from the Wireshark frame
    data.columns = headers

    # Export the pandas DataFrame as a CSV file
    data.to_csv(output_filename, index=False, sep=',')
```





In [3]:
!cp -rn "/content/drive/My Drive/AWID/AWID-CLS-R-Trn_with_headers.zip" .
!cp -rn "/content/drive/My Drive/AWID/AWID-CLS-R-Tst_with_headers.zip" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/utils.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/main.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/imports.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/Dataset/col_names.txt" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/class_report.py" .

!unzip AWID-CLS-R-Trn_with_headers.zip 
!unzip AWID-CLS-R-Tst_with_headers.zip 
!rm *.zip
!rm -rf sample_data/

Archive:  AWID-CLS-R-Trn_with_headers.zip
  inflating: AWID-CLS-R-Trn/AWID-CLS-R-Trn_with_headers.csv  
Archive:  AWID-CLS-R-Tst_with_headers.zip
  inflating: AWID-CLS-R-Tst/AWID-CLS-R-Tst_with_headers.csv  


***Imports***


> Import all necessary libraries.



In [0]:
from imports import *

***Read the data and replace the '?' string in the Pandas DataFrame with a NumPy NaN value***

In [0]:
train_path = '/content/AWID-CLS-R-Trn/AWID-CLS-R-Trn_with_headers.csv'
test_path = '/content/AWID-CLS-R-Tst/AWID-CLS-R-Tst_with_headers.csv'
train_df_raw = pd.read_csv(train_path, low_memory=False)
test_df_raw = pd.read_csv(test_path, low_memory=False)
train_df_raw.replace('?', np.nan, inplace=True)
test_df_raw.replace('?', np.nan, inplace=True)


In [24]:
print("train :",train_df_raw.shape)
print("test :",test_df_raw.shape)

train : (1795574, 155)
test : (575642, 155)


**prepare a list of colomns to drop**

In [40]:
display_all(train_df_raw.isnull().sum().sort_index() / len(train_df_raw))

class                                             0.000000
data.len                                          0.502914
frame.cap_len                                     0.000000
frame.dlt                                         1.000000
frame.ignored                                     0.000000
frame.interface_id                                0.000000
frame.len                                         0.000000
frame.marked                                      0.000000
frame.offset_shift                                0.000000
frame.time_delta                                  0.000000
frame.time_delta_displayed                        0.000000
frame.time_epoch                                  0.000000
frame.time_relative                               0.000000
radiotap.antenna                                  0.000434
radiotap.channel.freq                             0.000434
radiotap.channel.type.2ghz                        0.000434
radiotap.channel.type.5ghz                        0.0004

In [41]:
display_all(test_df_raw.isnull().sum().sort_index() / len(test_df_raw))

class                                             0.000000
data.len                                          0.454913
frame.cap_len                                     0.000000
frame.dlt                                         1.000000
frame.ignored                                     0.000000
frame.interface_id                                0.000000
frame.len                                         0.000000
frame.marked                                      0.000000
frame.offset_shift                                0.000000
frame.time_delta                                  0.000000
frame.time_delta_displayed                        0.000000
frame.time_epoch                                  0.000000
frame.time_relative                               0.000000
radiotap.antenna                                  0.000452
radiotap.channel.freq                             0.000452
radiotap.channel.type.2ghz                        0.000452
radiotap.channel.type.5ghz                        0.0004

**Drop these colomns**


> Each one of the dropped columns has a Nan rate of 1.
> Note that we need to drop them from both of training and validation sets.


In [0]:
del test_df_raw['frame.dlt']
del test_df_raw['wlan.qos.buf_state_indicated']
del test_df_raw['radiotap.mactime']
del test_df_raw['wlan.ba.bm']
del test_df_raw['wlan.ba.control.ackpolicy']
del test_df_raw['wlan.ba.control.cbitmap']
del test_df_raw['wlan.ba.control.multitid']
del test_df_raw['wlan.bar.compressed.tidinfo']
del test_df_raw['wlan.bar.type']
del test_df_raw['wlan_mgt.fixed.fragment']
del test_df_raw['wlan_mgt.fixed.sequence']
del test_df_raw['wlan_mgt.tcprep.link_mrg']
del test_df_raw['wlan_mgt.tcprep.trsmt_pow']


del train_df_raw['frame.dlt']
del train_df_raw['wlan.qos.buf_state_indicated']
del train_df_raw['radiotap.mactime']
del train_df_raw['wlan.ba.bm']
del train_df_raw['wlan.ba.control.ackpolicy']
del train_df_raw['wlan.ba.control.cbitmap']
del train_df_raw['wlan.ba.control.multitid']
del train_df_raw['wlan.bar.compressed.tidinfo']
del train_df_raw['wlan.bar.type']
del train_df_raw['wlan_mgt.fixed.fragment']
del train_df_raw['wlan_mgt.fixed.sequence']
del train_df_raw['wlan_mgt.tcprep.link_mrg']
del train_df_raw['wlan_mgt.tcprep.trsmt_pow']


In [28]:
train_df_raw.describe()

,frame.interface_id,frame.offset_shift,frame.time_epoch,frame.time_delta,frame.time_delta_displayed,frame.time_relative,frame.len,frame.cap_len,frame.marked,frame.ignored,radiotap.version,radiotap.pad,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.rate,radiotap.present.channel,radiotap.present.fhss,radiotap.present.dbm_antsignal,radiotap.present.dbm_antnoise,radiotap.present.lock_quality,radiotap.present.tx_attenuation,radiotap.present.db_tx_attenuation,radiotap.present.dbm_tx_power,radiotap.present.antenna,radiotap.present.db_antsignal,radiotap.present.db_antnoise,radiotap.present.rxflags,radiotap.present.xchannel,radiotap.present.mcs,radiotap.present.ampdu,radiotap.present.vht,radiotap.present.rtap_ns,radiotap.present.vendor_ns,radiotap.present.ext,radiotap.datarate,wlan.fc.version,wlan.fc.type,wlan.fc.subtype,wlan.fc.frag,wlan.fc.retry,wlan.fc.pwrmgt,wlan.fc.moredata,wlan.fc.protected,wlan.fc.order
count,1795574.0,1795574.0,1.795574e+06,1.795574e+06,1.795574e+06,1.795574e+06,1.795574e+06,1.795574e+06,1795574.0,1795574.0,1795574.0,1795574.0,1.795574e+06,1.795574e+06,1.795574e+06,1795574.0,1.795574e+06,1795574.0,1.795574e+06,1795574.0,1795574.0,1795574.0,1795574.0,1795574.0,1.795574e+06,1795574.0,1795574.0,1.795574e+06,1795574.0,1795574.0,1795574.0,1795574.0,1795574.0,1795574.0,1795574.0,1.795574e+06,1795574.0,1.795574e+06,1.795574e+06,1.795574e+06,1.795574e+06,1.795574e+06,1.795574e+06,1.795574e+06,1795574.0
mean,0.0,0.0,1.393663e+09,2.004691e-03,2.004691e-03,1.870295e+03,5.153151e+02,5.153151e+02,0.0,0.0,0.0,0.0,2.599435e+01,9.995656e-01,9.995656e-01,1.0,9.995656e-01,0.0,9.995656e-01,0.0,0.0,0.0,0.0,0.0,9.995656e-01,0.0,0.0,9.995656e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.513951e+01,0.0,1.272583e+00,8.499055e+00,3.748105e-04,1.271081e-01,2.325217e-02,1.772692e-03,4.932930e-01,0.0
std,0.0,0.0,8.674439e+02,6.379538e-03,6.379538e-03,8.674439e+02,6.649246e+02,6.649246e+02,0.0,0.0,0.0,0.0,2.708911e-01,2.083778e-02,2.083778e-02,0.0,2.083778e-02,0.0,2.083778e-02,0.0,0.0,0.0,0.0,0.0,2.083778e-02,0.0,0.0,2.083778e-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.373590e+01,0.0,8.198057e-01,4.173016e+00,1.935640e-02,3.330941e-01,1.507034e-01,4.206603e-02,4.999552e-01,0.0
min,0.0,0.0,1.393661e+09,1.000000e-06,1.000000e-06,2.427100e-02,4.000000e+01,4.000000e+01,0.0,0.0,0.0,0.0,1.300000e+01,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,0.0,0.0,1.393662e+09,3.150000e-04,3.150000e-04,1.027876e+03,4.000000e+01,4.000000e+01,0.0,0.0,0.0,0.0,2.600000e+01,1.000000e+00,1.000000e+00,1.0,1.000000e+00,0.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,1.000000e+00,0.0,0.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000e+00,0.0,1.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,0.0,0.0,1.393663e+09,5.490000e-04,5.490000e-04,1.902615e+03,1.120000e+02,1.120000e+02,0.0,0.0,0.0,0.0,2.600000e+01,1.000000e+00,1.000000e+00,1.0,1.000000e+00,0.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,1.000000e+00,0.0,0.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.400000e+01,0.0,2.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,0.0,0.0,1.393664e+09,1.113000e-03,1.113000e-03,2.524325e+03,1.552000e+03,1.552000e+03,0.0,0.0,0.0,0.0,2.600000e+01,1.000000e+00,1.000000e+00,1.0,1.000000e+00,0.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,1.000000e+00,0.0,0.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.400000e+01,0.0,2.000000e+00,1.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.0
max,0.0,0.0,1.393665e+09,3.502630e-01,3.502630e-01,3.599571e+03,1.570000e+03,1.570000e+03,0.0,0.0,0.0,0.0,2.600000e+01,1.000000e+00,1.000000e+00,1.0,1.000000e+00,0.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,1.000000e+00,0.0,0.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.400000e+01,0.0,2.000000e+00,1.300

In [43]:
train_df_raw['frame.cap_len'].describe()

count    1.795574e+06
mean     5.153151e+02
std      6.649246e+02
min      4.000000e+01
25%      4.000000e+01
50%      1.120000e+02
75%      1.552000e+03
max      1.570000e+03
Name: frame.cap_len, dtype: float64

In [44]:
display_all(train_df_raw.dtypes)

frame.interface_id                                  int64
frame.offset_shift                                float64
frame.time_epoch                                  float64
frame.time_delta                                  float64
frame.time_delta_displayed                        float64
frame.time_relative                               float64
frame.len                                           int64
frame.cap_len                                       int64
frame.marked                                        int64
frame.ignored                                       int64
radiotap.version                                    int64
radiotap.pad                                        int64
radiotap.length                                     int64
radiotap.present.tsft                               int64
radiotap.present.flags                              int64
radiotap.present.rate                               int64
radiotap.present.channel                            int64
radiotap.prese

***Data pre-processing***


> We need to treat the data before passing it to our model.



***Change any columns of strings in a panda's dataframe to a column of categorical values.***
 

> This applies the changes inplace.




In [0]:
str_to_cat(train_df_raw)

verify the changes

In [46]:
display_all(train_df_raw.dtypes)

frame.interface_id                                   int64
frame.offset_shift                                 float64
frame.time_epoch                                   float64
frame.time_delta                                   float64
frame.time_delta_displayed                         float64
frame.time_relative                                float64
frame.len                                            int64
frame.cap_len                                        int64
frame.marked                                         int64
frame.ignored                                        int64
radiotap.version                                     int64
radiotap.pad                                         int64
radiotap.length                                      int64
radiotap.present.tsft                                int64
radiotap.present.flags                               int64
radiotap.present.rate                                int64
radiotap.present.channel                             int

***Changes any columns of strings in df into categorical variables using trn as a template for the category codes***
 

> This applies the changes inplace too.




In [0]:
apply_cats(test_df_raw,train_df_raw)

***We're still not quite done !***
 
> For instance we have lots of missing values, which we can't pass directly to the model.




***We call  the function proc_df() to process a pandas dataframe !***
 

*   *proc_df*  takes a data frame df and splits off the response variable in our case the *class* colomn, and changes the df into an entirely numeric dataframe. For each column of df which is not in skip_flds nor in ignore_flds, na values are replaced by the median value of the column.
*    Three other funtions are called inside this function:
    *    ***numericalize()*** : Changes the column col from a categorical type to it's integer codes.
    *    ***fix_missing()*** : Fill missing data in a column of df with the median, and add a {name}_na column 
    which specifies if the data was missing.
    *    ***scale_vars()*** : Scale tha data.



In [0]:
x_train, y_train , nas , mapper = proc_df(train_df_raw, 'class', do_scale=True)
x_valid, y_valid , nas , _ = proc_df(test_df_raw,'class', na_dict=nas, do_scale=True, mapper=mapper)

verify again!

In [37]:
x_train['frame.cap_len'].describe()

count    1.795574e+06
mean     9.098042e-13
std      1.000000e+00
min     -7.148407e-01
25%     -7.148407e-01
50%     -6.065577e-01
75%      1.559102e+00
max      1.586173e+00
Name: frame.cap_len, dtype: float64

the max was 1.570000e+03	, now its is 1.58